In [52]:
from scipy.spatial import distance_matrix
import numpy as np
import pandas as pd

A = [[2,3],[4,2],[4,5],[6,6],[7,6],[8,8],[9,6]]
B = np.triu(distance_matrix(A,A))
B = np.around(B,2)
B

array([[0.  , 2.24, 2.83, 5.  , 5.83, 7.81, 7.62],
       [0.  , 0.  , 3.  , 4.47, 5.  , 7.21, 6.4 ],
       [0.  , 0.  , 0.  , 2.24, 3.16, 5.  , 5.1 ],
       [0.  , 0.  , 0.  , 0.  , 1.  , 2.83, 3.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 2.24, 2.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 2.24],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [22]:
np.where(B == np.min(B[np.nonzero(B)]))

(array([3]), array([4]))

In [23]:
B[3][4] = 0

In [24]:
B

array([[0.  , 2.24, 2.83, 5.  , 5.83, 7.81, 7.62],
       [0.  , 0.  , 3.  , 4.47, 5.  , 7.21, 6.4 ],
       [0.  , 0.  , 0.  , 2.24, 3.16, 5.  , 5.1 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 2.83, 3.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 2.24, 2.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 2.24],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [68]:
from scipy.spatial import distance_matrix
import numpy as np

X = pd.read_csv('cluster.dat', sep = ' ', header = None)
X = (X-X.min())/(X.max()-X.min()) #Normalizando os dados de entrada
X = X.values 

#A = [[2,3],[4,2],[4,5],[6,6],[7,6],[8,8],[9,6]]
#B = np.triu(distance_matrix(A,A))
#B = np.around(B,2)

A = X.tolist()
B = np.triu(distance_matrix(A,A))

clusters = {}
tamanho = 0
while(tamanho != len(A)):
    Indice_min = np.where(B == np.min(B[np.nonzero(B)]))
    x = Indice_min[0][0]
    y = Indice_min[1][0]
    
    novo_cluster = True
    for i in range(len(clusters)):
        if(A[x] in clusters[i] and A[y] not in clusters[i]):
            clusters[i].append(A[y])
            novo_cluster = False
        elif(A[y] in clusters[i] and A[x] not in clusters[i]):
            clusters[i].append(A[x])
            novo_cluster = False          
           
    if(novo_cluster):
        tamanho = len(clusters)
        clusters[tamanho] = []
        clusters[tamanho].extend([A[x],A[y]])
        
    B[x][y] = 0
    tamanho = sum(len(cluster) for cluster in clusters.values())
print(len(clusters))

177


In [69]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances #We can calculate this matrix using 2 for loops, 
#but this isn't that important to calculate so we directly use this
import matplotlib.pyplot as plt



get_ipython().magic('matplotlib inline')


def OwnHeirarchical(data, cutoff, linkage):

    distance_matrix = euclidean_distances(data, data) #Step 1 - Calculate distance matrix
    distance_matrix = np.tril(distance_matrix) #Step 2 - Since matrix is symmetric, we just keep lower triangle matrix
    distance_matrix[distance_matrix == 0] = np.inf #Step 3 - Replace 0 by inf, it makes it easy for us to extract minimum using min function
    df = pd.DataFrame(data=np.ones(data.shape[0])*np.inf) #Initialized a dataframe which will store which point is in which cluster
    if cutoff > distance_matrix.shape[0]: #If user provides impractical cut-off, cluster everthing into one cluster and not listen to user 
        cutoff = distance_matrix.shape[0]
    if linkage == 1: #This 1 means formula of single linkage will be used, it is explained ahead
        d = {} #This dictionary keeps record of which data points or cluster are merging, hence can be used to make a dendogram
        for i in range(0,cutoff):
            ij_min = np.unravel_index(distance_matrix.argmin(), distance_matrix.shape) #from the distance matrix, get the minimum distance
            #np.unravel_index gives us the position of minimum distance. e.g. (0,4) is where minimum value is present in matrix.
            #This is what we need as in Hierarchical clustering, we merge the two pairs with minimum distance
            if i == 0:
                df.iloc[ij_min[0]] = 0
                df.iloc[ij_min[1]] = 0
            else:
                try:
                    a = int(df.iloc[ij_min[0]])
                except:
                    df.iloc[ij_min[0]] = i
                    a = i
                try:
                    b = int(df.iloc[ij_min[1]])
                except:
                    df.iloc[ij_min[1]] = i
                    b = i
                df[(df[0]==a) | (df[0]==b)] = i
            d[i] = ij_min
            #The if, else code till here is just filling the dataframe as the two points/clusters combine.
            #So, for example if 1 and 2 combines, dataframe will have 1 : 0, 2 : 0. Which means point 1 and 2 both are in same cluster (0th cluster)
            for j in range(0, ij_min[0]):
                #we want to ignore the diagonal, and diagonal is 0. We replaced 0 by infinte. 
                #So this if condition will skip diagonals
                if np.isfinite(distance_matrix[ij_min[0]][j]) and np.isfinite(distance_matrix[ij_min[1]][j]):
                    #after two points/cluster are linked, to calculate new distance we take minimum distance for single linkage
                    distance_matrix[ij_min[1]][j] = min(distance_matrix[ij_min[0]][j], distance_matrix[ij_min[1]][j])
            # To avoid the combined data points/cluster in further calculations, we make them infinte.
            #Our if loop above this, will therefore skip the infinite record entries.
            distance_matrix[ij_min[0]] = np.inf
        return d, df[0].to_numpy()
    elif linkage == 2:
        d_complete = {}
        for i in range(0,cutoff):
            ij_min = np.unravel_index(distance_matrix.argmin(), distance_matrix.shape)
            if i == 0:
                df.iloc[ij_min[0]] = 0
                df.iloc[ij_min[1]] = 0
            else:
                try:
                    a = int(df.iloc[ij_min[0]])
                except:
                    df.iloc[ij_min[0]] = i
                    a = i
                try:
                    b = int(df.iloc[ij_min[1]])
                except:
                    df.iloc[ij_min[1]] = i
                    b = i
                df[(df[0]==a) | (df[0]==b)] = i
            d_complete[i] = ij_min
            for j in range(0, ij_min[0]):
                if np.isfinite(distance_matrix[ij_min[0]][j]) and np.isfinite(distance_matrix[ij_min[1]][j]):
                    #after two points/cluster are linked, to calculate new distance we take maximum distance for complete linkage
                    distance_matrix[ij_min[1]][j] = max(distance_matrix[ij_min[0]][j], distance_matrix[ij_min[1]][j])
            distance_matrix[ij_min[0]] = np.inf
        return d_complete, df[0].as_matrix()
    elif linkage == 3:
        d_average = {}
        for i in range(0,cutoff):
            ij_min = np.unravel_index(distance_matrix.argmin(), distance_matrix.shape)
            if i == 0:
                df.iloc[ij_min[0]] = 0
                df.iloc[ij_min[1]] = 0
            else:
                try:
                    a = int(df.iloc[ij_min[0]])
                except:
                    df.iloc[ij_min[0]] = i
                    a = i
                try:
                    b = int(df.iloc[ij_min[1]])
                except:
                    df.iloc[ij_min[1]] = i
                    b = i
                df[(df[0]==a) | (df[0]==b)] = i
            d_average[i] = ij_min
            for j in range(0, ij_min[0]):
                if np.isfinite(distance_matrix[ij_min[0]][j]) and np.isfinite(distance_matrix[ij_min[1]][j]):
                    #after two points/cluster are linked, to calculate new distance we take average distance for average linkage
                    distance_matrix[ij_min[1]][j] = (distance_matrix[ij_min[0]][j] + distance_matrix[ij_min[1]][j])/2.0          
            distance_matrix[ij_min[0]] = np.inf
        return d_average, df[0].as_matrix()
    
d, target = OwnHeirarchical(X, 147, 1)

In [77]:
d

{0: (400, 399),
 1: (536, 535),
 2: (80, 79),
 3: (333, 332),
 4: (413, 412),
 5: (476, 475),
 6: (496, 495),
 7: (297, 296),
 8: (458, 457),
 9: (265, 264),
 10: (420, 419),
 11: (355, 354),
 12: (398, 397),
 13: (320, 319),
 14: (433, 432),
 15: (308, 307),
 16: (12, 11),
 17: (555, 554),
 18: (449, 448),
 19: (300, 299),
 20: (287, 286),
 21: (221, 216),
 22: (463, 462),
 23: (285, 283),
 24: (324, 323),
 25: (274, 273),
 26: (390, 389),
 27: (212, 211),
 28: (298, 296),
 29: (542, 541),
 30: (448, 447),
 31: (109, 108),
 32: (241, 240),
 33: (359, 356),
 34: (252, 243),
 35: (75, 74),
 36: (159, 158),
 37: (336, 334),
 38: (223, 215),
 39: (284, 283),
 40: (427, 426),
 41: (477, 475),
 42: (312, 311),
 43: (422, 421),
 44: (486, 485),
 45: (182, 181),
 46: (116, 115),
 47: (232, 231),
 48: (347, 346),
 49: (47, 46),
 50: (43, 42),
 51: (65, 60),
 52: (126, 122),
 53: (124, 123),
 54: (267, 266),
 55: (39, 38),
 56: (349, 348),
 57: (571, 570),
 58: (506, 505),
 59: (97, 96),
 60: (

In [75]:
target[target == 140] = 0
target[target == 144] = 1
target[target == 146] = 2

In [76]:
target

array([ inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        16.,  16.,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,
        inf,  inf,  inf, 117., 117.,  inf,  inf,  inf, 108., 108.,  inf,
        inf,  inf,  inf,  inf,  83.,  83.,  83.,  83.,  inf,  50.,  50.,
        inf,  inf,  89.,  89.,  89.,  inf,  inf,  inf,  inf,  inf,  inf,
        inf,  inf,  inf,  inf,  inf, 138., 138.,  inf, 138., 138., 138.,
        inf,  inf,  inf,  inf,  inf,  inf,  inf,  inf,  35.,  35.,  inf,
        inf,  inf,   2.,   2.,  inf,  inf,  inf,  inf,   1.,   1.,  inf,
        inf,  inf,  inf,  81.,  81.,   2.,   2.,  inf, 103., 103., 103.,
        inf, 121., 121., 121.,  inf,  inf,  inf,  inf,  inf,  31.,  31.,
        inf,  inf, 126., 126.,  inf,  46.,  46.,  inf,  inf,  inf,  62.,
        62.,  52.,  53.,  53.,  inf,  52.,  inf,  inf,  inf,  inf,  inf,
        inf, 122., 122.,  96.,  96.,  inf,  80.,  80.,  inf,  inf,  inf,
        inf,  inf,  inf,  inf,  inf,  inf,  inf,  i